In [ ]:
# !pip install 
# import torch.hubimport os
# import sys
# current = os.path.dirname(os.path.realpath("inversion-stylegan2.ipynb"))
# parent = os.path.dirname(current)
# sys.path.append(parent)

In [ ]:
!git clone https://github.com/yossigandelsman/rosetta_neurons.git

In [ ]:
%cd rosetta_neurons

In [ ]:
import torch
device = torch.device('cuda:0')
dino = torch.hub.load('facebookresearch/dino:main', 'dino_resnet50').to(device)

In [ ]:
#@title Requirements
!pip install -r requirements.txt
!pip install transformers
!pip install openai-clip
!pip install einops
!pip install pytorch_pretrained_biggan
!pip install Ninja
!pip install timm==0.4.12
!pip install dill

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision
from torchvision.models import resnet50
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import clip
from PIL import Image
import requests
import torch.hub
import time
import pickle
import math

from match_utils import matching, stats, proggan, nethook, dataset, loading, plotting, layers, models

In [ ]:
!pip install gdown==4.6.3
!gdown "https://drive.google.com/uc?export=download&id=1nB5RQPrWRfcLI7yHhKKkf4gjxrJW7HCj" -O 'stylegan2-cat-config-f.pt'

In [ ]:
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal)


gan, gan_layers = models.load_gan('stylegan2-lsun_cat', path='/kaggle/working/rosetta_neurons/', device=device)    
dino, dino_layers = models.load_discr('dino', path='models/', device=device)

In [ ]:
    ganlayers, dinolayers = layers.get_layers(gan,gan_layers, dino, dino_layers,"stylegan2-lsun_cat", "dino", device)

In [ ]:
import os
table, gan_stats, dino_stats = loading.load_stats("/kaggle/input/style-gan2-lsun-cat/rosetta_neurons/matches/stylegan2-lsun_cat/dino", 
                                                  device)

save_path = "/kaggle/working/rosetta_neurons/inversion_images_stylegan2/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

### Best Buddies

In [ ]:
match_scores,_ = torch.max(table,1)

In [ ]:
gan_matches = torch.argmax(table,1)
dino_matches = torch.argmax(table,0)

In [ ]:
perfect_matches = []
perfect_match_scores = []
dino_perfect_matches = []
num_perfect_matches = 0 
for i in range(table.shape[0]):
    gan_match = gan_matches[i].item()
    dino_match = dino_matches[gan_match].item()
    if dino_match == i:
        #print(i)
        num_perfect_matches+=1
        perfect_matches.append(i)
        dino_perfect_matches.append(gan_match)
        perfect_match_scores.append(match_scores[i])
        
print(num_perfect_matches)
print(num_perfect_matches/table.shape[0])

In [ ]:
gan = nethook.InstrumentedModel(gan)
gan.retain_layers(gan_layers, detach = False)

dino = nethook.InstrumentedModel(dino)
dino.retain_layers(dino_layers)

In [ ]:
for i, unit in enumerate(perfect_matches):
    perfect_matches[i] = layers.find_act(perfect_matches[i], ganlayers)#,all_gan_layers)

In [ ]:
for i, unit in enumerate(dino_perfect_matches):
    dino_perfect_matches[i] = layers.find_act(dino_perfect_matches[i], dinolayers)#,all_dino_layers)

In [ ]:
from scipy.stats import truncnorm
def truncate_noise(size, truncation):
    '''
    Function for creating truncated noise vectors: Given the dimensions (n_samples, z_dim)
    and truncation value, creates a tensor of that shape filled with random
    numbers from the truncated normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        truncation: the truncation value, a non-negative scalar
    '''
    
    truncated_noise = truncnorm.rvs(-1*truncation, truncation, size=size)
    
    return torch.Tensor(truncated_noise)

In [ ]:
z1 = truncate_noise((1,512), 1).to(device)#
from torch.autograd import Variable

In [ ]:
z = Variable(z1.clone(), requires_grad=True)

with torch.no_grad():
    mean_latent = gan.model.mean_latent(4096)


In [ ]:
reg = torch.zeros((1,512)).to(device).detach()

In [ ]:
def show_gan_im(gan_im):
    im = (gan_im+1)/2
    im = torch.permute(im[0],(1,2,0)).detach().cpu()
    plt.imshow(im)
    plt.show()
    #plt.savefig( "cat1.png")

In [ ]:
c_dataset = []
for iii in range(1):
    c_dataset.append(mean_latent)

In [ ]:
c=gan.model.mean_latent(4096).view(-1)

In [ ]:
print(c.shape)

In [ ]:
img, _  = gan([z], 0.7, c)
print(img.shape)
show_gan_im(img)

In [ ]:
#real_im = Image.open("/home/amildravid/bigGAN-DINO_swap/val_im/ILSVRC2012_val_00028617-_1_.jpg")
#real_im = Image.open("/home/amildravid/bigGAN-DINO_swap/golden_retriever/real/ILSVRC2012_val_00001112.jpg")
#real_im = Image.open("/home/amildravid/bigActivation_Matching/val_im/ILSVRC2012_val_00006981.jpg")
real_im = Image.open("/kaggle/input/trial-good/download.jpg")
#real_im = Image.open("/home/amildravid/bigGAN-DINO_swap/golden_retriever/sketch/sketch_7.jpg")
real_im

In [ ]:
!mkdir -p imgs/dir0
!mkdir -p imgs/dir1

In [ ]:
real_im = torchvision.transforms.ToTensor()(real_im).unsqueeze(0).to(device)
#real_im = torchvision.transforms.RandomResizedCrop(256)(real_im)
real_im = torch.nn.functional.interpolate(real_im, size = (256,256), mode = "bicubic")
dino_real_im = torch.nn.functional.interpolate(real_im, size = (256,256), mode = "bicubic")
dino_real_im = torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(dino_real_im)

In [ ]:
!pwd

In [ ]:
plt.imshow(torch.permute(real_im[0], (1,2,0)).cpu())
plt.axis('off')
plt.savefig( "imgs/dir0/image_1.jpg")
plt.show()

In [ ]:
dino(dino_real_im)
dino_activs =  matching.store_activs(dino, dino_layers)
#normalize
eps = 0.00001
for i,_ in enumerate(dino_activs):
    dino_activs[i] = (dino_activs[i]-dino_stats[i][0])/(dino_stats[i][1]+eps)

In [ ]:
dino_perfect_activs = []
for idx in dino_perfect_matches:
    dino_perfect_activs.append(dino_activs[idx[0]][:,idx[1],:,:].unsqueeze(0))

### Pixel Loss

In [ ]:
optim = torch.optim.Adam([z], lr=0.01, betas=(0.5, 0.999))  

In [ ]:
for epoch in range(0,200):
    
    optim.zero_grad()
    sample,_ = gan([z],0.7,c)
    im = (sample+1)/2
    if(epoch%20==0):
        print(im.shape)
    
    loss = torch.mean((im-real_im)**2)
    
    
    
    if(epoch%20==0):
        print("E:", epoch+1, "loss:", loss.item())
    loss.backward()
    optim.step()
    if(epoch%20==0):
        show_gan_im(sample)
    
    
    im = torch.permute(sample[0],(1,2,0)).detach().cpu().numpy()
    im = (im+1)/2
    
    
    if epoch<=9:
        file_name = "00"+str(epoch)+".jpg"
    elif epoch<=99:
        file_name = "0"+str(epoch)+".jpg"
    else: 
        file_name = str(epoch)+".jpg"
   # plt.imsave(save_path+file_name, im)  
plt.imshow(im)
plt.axis('off')
plt.savefig( f"pixel_loss_{file_name}")
plt.show()

# Activation Loss

In [ ]:
optim = torch.optim.Adam([z], lr=0.01, betas=(0.5, 0.999))  

In [ ]:
for epoch in range(0,200):
    
    optim.zero_grad()
    sample,_ = gan([z],0.7,c)
    
    
    
    gan_activs = matching.store_activs(gan, gan_layers)
    
    
    #normalize all activations
    eps = 0.00001
    for i,_ in enumerate(gan_activs):
        gan_activs[i] = (gan_activs[i]-gan_stats[i][0])/(gan_stats[i][1]+eps)
        
    
    gan_perfect_activs = []
    for idx in perfect_matches:
        gan_perfect_activs.append(gan_activs[idx[0]][:,idx[1],:,:])
    
    
    loss = 0
    losses = []
    for i, _ in enumerate(gan_perfect_activs): 
        map_size = max((gan_perfect_activs[i].shape[1], dino_perfect_activs[i].shape[1]))
        gan_activ_new = torch.nn.Upsample(size=(map_size,map_size), mode='bilinear')(gan_perfect_activs[i].unsqueeze(0))
        dino_activ_new = torch.nn.Upsample(size=(map_size,map_size), mode='bilinear')(dino_perfect_activs[i])   
        #loss += torch.einsum('aixy,ajxy->ij', gan_activ_new,dino_activ_new)/(map_size**2)
        prod = torch.einsum('aixy,ajxy->ij', gan_activ_new,dino_activ_new)
        div1 = torch.sum(gan_activ_new**2)
        div2 = torch.sum(dino_activ_new**2)
        corr = prod/torch.sqrt(div1*div2)
        loss += corr
        losses.append(corr)
        #loss += torch.mean((gan_activ_new-dino_activ_new)**2)
        
        
        
    loss *= -1 
    #regularization = 50*torch.mean((z-reg)**2)
    #loss +=  regularization
    if(epoch%20==0):
        print("E:", epoch+1, "loss:", loss.item())
    loss.backward()
    optim.step()
    if(epoch%20==0):
        show_gan_im(sample)
    im = (sample+1)/2
    
    im = torch.permute(sample[0],(1,2,0)).detach().cpu().numpy()
    im = (im+1)/2
    
    
    if epoch<=9:
        file_name = "00"+str(epoch)+".jpg"
    elif epoch<=99:
        file_name = "0"+str(epoch)+".jpg"
    else: 
        file_name = str(epoch)+".jpg"
    
    #plt.imsave(save_path+file_name, im)
plt.imshow(im)
plt.axis('off')
plt.savefig( f"activation_loss_{file_name}")
plt.show()

### Both Losses

In [ ]:
optim = torch.optim.Adam([z], lr=0.01, betas=(0.5, 0.999))  

In [ ]:
for epoch in range(0,200):
    
    optim.zero_grad()
    sample,_ = gan([z],0.7,c)
    
    im = (sample+1)/2
    pixel_loss = torch.mean((im-real_im)**2)
    
    
    if(epoch%20==0):
        print(pixel_loss)
    
    
    gan_activs = matching.store_activs(gan, gan_layers)
    
    
    #normalize all activations
    eps = 0.00001
    for i,_ in enumerate(gan_activs):
        gan_activs[i] = (gan_activs[i]-gan_stats[i][0])/(gan_stats[i][1]+eps)
        
    
    gan_perfect_activs = []
    for idx in perfect_matches:
        gan_perfect_activs.append(gan_activs[idx[0]][:,idx[1],:,:])
    
    
    loss = 0
    losses = []
    for i, _ in enumerate(gan_perfect_activs): 
        map_size = max((gan_perfect_activs[i].shape[1], dino_perfect_activs[i].shape[1]))
        gan_activ_new = torch.nn.Upsample(size=(map_size,map_size), mode='bilinear')(gan_perfect_activs[i].unsqueeze(0))
        dino_activ_new = torch.nn.Upsample(size=(map_size,map_size), mode='bilinear')(dino_perfect_activs[i])   
        #loss += torch.einsum('aixy,ajxy->ij', gan_activ_new,dino_activ_new)/(map_size**2)
        prod = torch.einsum('aixy,ajxy->ij', gan_activ_new,dino_activ_new)
        div1 = torch.sum(gan_activ_new**2)
        div2 = torch.sum(dino_activ_new**2)
        corr = prod/torch.sqrt(div1*div2)
        loss += corr
        losses.append(corr)
        #loss += torch.mean((gan_activ_new-dino_activ_new)**2)
        
        
        
    loss *= -1 
    if(epoch%20==0):
        print(loss)
    loss += 100*pixel_loss
    
    
    #regularization = 100*torch.mean((z-reg)**2)
        #loss +=  regularization
    if(epoch%20==0):
        print("E:", epoch+1, "loss:", loss.item())
    loss.backward()
    optim.step()
    if(epoch%20==0):
        show_gan_im(sample)
    im = (sample+1)/2
    
    im = torch.permute(sample[0],(1,2,0)).detach().cpu().numpy()
    im = (im+1)/2
    
    
    if epoch<=9:
        file_name = "00"+str(epoch)+".jpg"
    elif epoch<=99:
        file_name = "0"+str(epoch)+".jpg"
    else: 
        file_name = str(epoch)+".jpg"
    
    #plt.imsave(save_path+file_name, im)
plt.imshow(im)
plt.axis('off')
plt.savefig( f"imgs/dir1/image_1.jpg")
plt.show()

In [ ]:
!pip install lpips

In [ ]:
!ls /opt/conda/lib/python3.10/site-packages/lpips

In [ ]:
code = '''
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
# from torch.autograd import Variable

from lpips.trainer import *
from lpips.lpips import *

def normalize_tensor(in_feat,eps=1e-10):
    norm_factor = torch.sqrt(torch.sum(in_feat**2,dim=1,keepdim=True))
    return in_feat/(norm_factor+eps)

def l2(p0, p1, range=255.):
    return .5*np.mean((p0 / range - p1 / range)**2)

def psnr(p0, p1, peak=255.):
    return 10*np.log10(peak**2/np.mean((1.*p0-1.*p1)**2))

def dssim(p0, p1, range=255.):
    from skimage.measure import compare_ssim
    return (1 - compare_ssim(p0, p1, data_range=range, multichannel=True)) / 2.

def tensor2np(tensor_obj):
    # change dimension of a tensor object into a numpy array
    return tensor_obj[0].cpu().float().numpy().transpose((1,2,0))

def np2tensor(np_obj):
     # change dimenion of np array into tensor array
    return torch.Tensor(np_obj[:, :, :, np.newaxis].transpose((3, 2, 0, 1)))

def tensor2tensorlab(image_tensor,to_norm=True,mc_only=False):
    # image tensor to lab tensor
    from skimage import color

    img = tensor2im(image_tensor)
    img_lab = color.rgb2lab(img)
    if(mc_only):
        img_lab[:,:,0] = img_lab[:,:,0]-50
    if(to_norm and not mc_only):
        img_lab[:,:,0] = img_lab[:,:,0]-50
        img_lab = img_lab/100.

    return np2tensor(img_lab)

def tensorlab2tensor(lab_tensor,return_inbnd=False):
    from skimage import color
    import warnings
    warnings.filterwarnings("ignore")

    lab = tensor2np(lab_tensor)*100.
    lab[:,:,0] = lab[:,:,0]+50

    rgb_back = 255.*np.clip(color.lab2rgb(lab.astype('float')),0,1)
    if(return_inbnd):
        # convert back to lab, see if we match
        lab_back = color.rgb2lab(rgb_back.astype('uint8'))
        mask = 1.*np.isclose(lab_back,lab,atol=2.)
        mask = np2tensor(np.prod(mask,axis=2)[:,:,np.newaxis])
        return (im2tensor(rgb_back),mask)
    else:
        return im2tensor(rgb_back)

def load_image(path):
    if(path[-3:] == 'dng'):
        import rawpy
        with rawpy.imread(path) as raw:
            img = raw.postprocess()
    elif(path[-3:]=='bmp' or path[-3:]=='jpg' or path[-3:]=='png' or path[-4:]=='jpeg'):
        import cv2
        return cv2.imread(path)[:,:,::-1]
    else:
        import matplotlib.pyplot as plt        
        img = (255*plt.imread(path)[:,:,:3]).astype('uint8')

    return img

def tensor2im(image_tensor, imtype=np.uint8, cent=1., factor=255./2.):
    image_numpy = image_tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + cent) * factor
    return image_numpy.astype(imtype)

def im2tensor(image, imtype=np.uint8, cent=1., factor=255./2.):
    return torch.Tensor((image / factor - cent)
                        [:, :, :, np.newaxis].transpose((3, 2, 0, 1)))

def tensor2vec(vector_tensor):
    return vector_tensor.data.cpu().numpy()[:, :, 0, 0]


def voc_ap(rec, prec, use_07_metric=False):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap'''

In [ ]:
with open("/opt/conda/lib/python3.10/site-packages/lpips/__init__.py","w") as f:
    f.write(code)

In [ ]:
!python calc_metrics.py  --dir0 /kaggle/working/rosetta_neurons/imgs/dir0 --dir1 /kaggle/working/rosetta_neurons/imgs/dir1 -o imgs_dist.txt --use_gpu --version 0.1